In [5]:
import sys
from os.path import dirname
sys.path.append(dirname(rf'C:\Projects\trading'))
sys.path.append(dirname(rf'C:\Projects\trading\indicators'))

In [39]:
from os import path

import pandas as pd
import numpy as np
from ta.trend import DPOIndicator, adx_pos, adx_neg, cci, macd_diff, EMAIndicator
from ta.volume import chaikin_money_flow, acc_dist_index, negative_volume_index
from ta.momentum import ROCIndicator, williams_r, stoch_signal, RSIIndicator
from ta.volatility import average_true_range
import pygad
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import indicators

In [38]:
df = pd.read_csv(path.join("D:\\Trading\\raw_data\\special", "SP500.csv"), parse_dates=True, index_col="Date")
data_df = pd.DataFrame(index=df.index)
data_df["DPO"] = DPOIndicator(close=df["Close"]).dpo()
data_df["CMF"] = chaikin_money_flow(high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"])
data_df["ADI"] = acc_dist_index(high=df["High"], low=df["Low"], close=df["Close"], volume=df["Volume"])
data_df["ROC"] = ROCIndicator(close=df["Close"]).roc()
data_df["NVI"] = negative_volume_index(close=df["Close"], volume=df["Volume"])
data_df["DIU"] = adx_pos(high=df["High"], low=df["Low"], close=df["Close"])
data_df["DID"] = adx_neg(high=df["High"], low=df["Low"], close=df["Close"])
data_df["WILLR"] = williams_r(high=df["High"], low=df["Low"], close=df["Close"])
data_df["CCI"] = cci(high=df["High"], low=df["Low"], close=df["Close"])
data_df["STOH"] = stoch_signal(high=df["High"], low=df["Low"], close=df["Close"])
data_df["MACD"] = macd_diff(close=df["Close"])
data_df["RSI"] = RSIIndicator(close=df["Close"]).rsi()
data_df["ATR"] = average_true_range(high=df["High"], low=df["Low"], close=df["Close"])
data_df["APO"] = indicators.apo(close=df["Close"])
data_df["BIAS"] = indicators.bias(close=df["Close"])
data_df["DEMA"] = indicators.dema(close=df["Close"])
data_df["HULL"] = indicators.hull_ma(close=df["Close"])
data_df["MFM"] = indicators.mfm(df)
data_df["RVI"] = indicators.rvi(df)
data_df["Supertrend"] = indicators.supertrend(df)
data_df["EMA20"] = EMAIndicator(close=df["Close"], window=20).ema_indicator()
data_df["EMA50"] = EMAIndicator(close=df["Close"], window=50).ema_indicator()
data_df["EMA100"] = EMAIndicator(close=df["Close"], window=100).ema_indicator()

for ind in data_df.columns:
    for i in range(1, 5):
        data_df[f"{ind}_{i}"] = data_df[ind].shift(periods=i)

data_df["target_5"] = 0
pct_change = df["Close"].pct_change(periods=4).shift(-4)
insign_val = 0.02
strong_val = 0.04
data_df.loc[pct_change >= strong_val, "target_5"] = 2
data_df.loc[(pct_change < strong_val) & (pct_change >= insign_val), "target_5"] = 1
data_df.loc[(pct_change > -strong_val) & (pct_change <= -insign_val), "target_5"] = -1
data_df.loc[pct_change <= -strong_val, "target_5"] = -2

data_df["target_3"] = 0
pct_change = df["Close"].pct_change(periods=2).shift(-2)
insign_val = 0.01
strong_val = 0.025
data_df.loc[pct_change >= strong_val, "target_3"] = 2
data_df.loc[(pct_change < strong_val) & (pct_change >= insign_val), "target_3"] = 1
data_df.loc[(pct_change > -strong_val) & (pct_change <= -insign_val), "target_3"] = -1
data_df.loc[pct_change <= -strong_val, "target_3"] = -2

data_df.dropna(inplace=True)

train_data = data_df[:"2020-01-01"]
train_X = train_data.drop(["target_5", "target_3"], axis=1)
test_data = data_df["2020-01-01":]
test_X = test_data.drop(["target_5", "target_3"], axis=1)
indicators_pull = train_X.columns

C:\Users\Dell\AppData\Local\Temp\ipykernel_17120\2768094284.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[f"{ind}_{i}"] = data_df[ind].shift(periods=i)
C:\Users\Dell\AppData\Local\Temp\ipykernel_17120\2768094284.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[f"{ind}_{i}"] = data_df[ind].shift(periods=i)
C:\Users\Dell\AppData\Local\Temp\ipykernel_17120\2768094284.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [32]:
test_data["target_5"].value_counts() / test_data.shape[0]

 0    0.670487
 1    0.141356
-1    0.094556
-2    0.047755
 2    0.045845
Name: target_5, dtype: float64

In [33]:
test_data["target_3"].value_counts() / test_data.shape[0]

 0    0.501433
 1    0.221585
-1    0.153773
-2    0.063992
 2    0.059217
Name: target_3, dtype: float64

In [42]:
clf = RandomForestClassifier(max_depth=6, class_weight={0: 0.2, 1: 1, -1: 1, 2: 4, -2: 4}, random_state=42)
np.random.seed(42)
init_pop_p = 0.33
initial_pop = np.random.choice([1, 0], p=[init_pop_p, 1-init_pop_p], size=(20, train_X.shape[1]))

def fitness_function(ga_inst, solution, solution_idx):
    selected_indicators = indicators_pull[solution.astype(bool)]
    clf.fit(X=train_X[selected_indicators], y=train_data["target_5"])
    predicts = clf.predict(X=test_X[selected_indicators])
    acc = accuracy_score(y_true=test_data["target_5"], y_pred=predicts)
    return acc


ga_instance = pygad.GA(num_generations=300,
                       num_parents_mating=2,
                       fitness_func=fitness_function,
                       initial_population=initial_pop,
                       num_genes=train_X.shape[1],
                       parent_selection_type="rws",
                       keep_elitism=2,
                       crossover_type="uniform",
                       mutation_type="random",
                       mutation_probability=0.02,
                       gene_space=[1, 0],
                       random_seed=42)
ga_instance.run()
solution, solution_fitness, solution_idx = ga_instance.best_solution()
selected_indicators = indicators_pull[solution.astype(bool)]
clf.fit(X=train_X[selected_indicators], y=train_data["target_5"])
predicts = clf.predict(X=test_X[selected_indicators])
predicts = pd.Series(predicts)
print(f"Accuracy: {solution_fitness}")
print(predicts.value_counts() / predicts.shape[0])
print(f"Indicators:")
print(indicators_pull[solution.astype(bool)])

Accuracy: 0.6494746895893028
0    0.873926
1    0.104107
2    0.021968
dtype: float64
Indicators:
Index(['ADI', 'DIU', 'DID', 'DEMA', 'RVI', 'Supertrend', 'EMA100', 'DPO_3',
       'DPO_4', 'CMF_1', 'CMF_2', 'CMF_4', 'ADI_1', 'ADI_3', 'ADI_4', 'ROC_1',
       'ROC_4', 'NVI_2', 'NVI_3', 'DIU_2', 'DID_1', 'DID_3', 'WILLR_2',
       'WILLR_3', 'CCI_1', 'STOH_2', 'STOH_3', 'MACD_3', 'MACD_4', 'BIAS_2',
       'DEMA_2', 'DEMA_3', 'HULL_3', 'HULL_4', 'MFM_3', 'Supertrend_1',
       'Supertrend_2', 'EMA50_1', 'EMA50_4', 'EMA100_1', 'EMA100_4'],
      dtype='object')


In [43]:
clf2 = RandomForestClassifier(max_depth=6, class_weight={0: 0.2, 1: 1, -1: 1, 2: 4, -2: 4}, random_state=42)
np.random.seed(42)
init_pop_p = 0.33
initial_pop = np.random.choice([1, 0], p=[init_pop_p, 1-init_pop_p], size=(20, train_X.shape[1]))

def fitness_function2(ga_inst, solution, solution_idx):
    selected_indicators = indicators_pull[solution.astype(bool)]
    clf2.fit(X=train_X[selected_indicators], y=train_data["target_3"])
    predicts = clf2.predict(X=test_X[selected_indicators])
    acc = accuracy_score(y_true=test_data["target_3"], y_pred=predicts)
    return acc


ga_instance2 = pygad.GA(num_generations=200,
                       num_parents_mating=2,
                       fitness_func=fitness_function2,
                       initial_population=initial_pop,
                       num_genes=train_X.shape[1],
                       parent_selection_type="rws",
                       keep_elitism=2,
                       crossover_type="uniform",
                       mutation_type="random",
                       mutation_probability=0.02,
                       gene_space=[1, 0],
                       random_seed=42)
ga_instance2.run()
solution, solution_fitness, solution_idx = ga_instance2.best_solution()

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- APO
- APO_4
- BIAS
- BIAS_1
- BIAS_4
- ...
Feature names seen at fit time, yet now missing:
- ADI
- ADI_1
- ADI_3
- CCI_1
- CMF_4
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 43 features, but RandomForestClassifier is expecting 41 features as input.

In [44]:
selected_indicators = indicators_pull[solution.astype(bool)]
clf2.fit(X=train_X[selected_indicators], y=train_data["target_3"])
predicts = clf2.predict(X=test_X[selected_indicators])
predicts = pd.Series(predicts)
print(f"Accuracy: {solution_fitness}")
print(predicts.value_counts() / predicts.shape[0])
print(f"Indicators:")
print(indicators_pull[solution.astype(bool)])

Accuracy: 0.4785100286532951
 0    0.594078
 1    0.191022
-2    0.135626
 2    0.068768
-1    0.010506
dtype: float64
Indicators:
Index(['ROC', 'DIU', 'STOH', 'APO', 'BIAS', 'HULL', 'Supertrend', 'EMA50',
       'EMA100', 'DPO_1', 'DPO_2', 'DPO_4', 'CMF_1', 'CMF_2', 'ADI_4', 'ROC_4',
       'NVI_2', 'NVI_3', 'DIU_2', 'DIU_3', 'DID_4', 'WILLR_2', 'WILLR_3',
       'CCI_3', 'STOH_3', 'MACD_1', 'MACD_4', 'RSI_2', 'APO_4', 'BIAS_1',
       'BIAS_2', 'BIAS_4', 'DEMA_1', 'HULL_2', 'HULL_3', 'MFM_3', 'RVI_4',
       'Supertrend_2', 'Supertrend_3', 'EMA20_2', 'EMA20_4', 'EMA50_3',
       'EMA100_4'],
      dtype='object')
